<a href="https://colab.research.google.com/github/m-zayed5722/Miscellaneous-Projects/blob/main/QueryCraft_Lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import duckdb
import pandas as pd
import re
import sqlparse
from sqlparse.sql import Statement
from sqlparse.tokens import Keyword
import matplotlib.pyplot as plt

In [2]:
import random
from datetime import date, timedelta

random.seed(7)

teams = ["Platform", "Payments", "Core", "Mobile", "Data", "Security"]
directors = ["Chen", "Singh", "Garcia", "Patel", "Nguyen"]
platforms = ["iOS", "Android", "Web", "Backend"]
statuses = ["planning", "in_progress", "in_production", "paused", "retired"]
modernization = ["rehost", "refactor", "rebuild", "replace", "none"]

def rand_date(start_days_ago=900, end_days_ago=0):
    d = date.today() - timedelta(days=random.randint(end_days_ago, start_days_ago))
    return d.isoformat()

rows = []
for app_id in range(1001, 1121):
    row = {
        "app_id": app_id,
        "app_name": f"App-{app_id}",
        "team": random.choice(teams),
        "director": random.choice(directors),
        "platform": random.choice(platforms),
        "overall_status": random.choice(statuses),
        "ryg_status": random.choice(["red", "yellow", "green"]),
        "planned_launch_date": rand_date(700, 0),
        "modernization_type": random.choice(modernization),
        "monthly_active_users": random.randint(200, 200000),
        "incidents_last_30d": random.randint(0, 18),
        "cost_usd_per_month": random.randint(1200, 85000),
    }
    rows.append(row)

df = pd.DataFrame(rows)
df.head()

,app_id,app_name,team,director,platform,overall_status,ryg_status,planned_launch_date,modernization_type,monthly_active_users,incidents_last_30d,cost_usd_per_month
0,1001,App-1001,Core,Singh,Backend,planning,red,2024-07-18,rehost,96063,18,8802
1,1002,App-1002,Data,Singh,iOS,planning,yellow,2024-11-15,rehost,63288,2,73426
2,1003,App-1003,Mobile,Chen,iOS,in_progress,green,2024-04-15,none,16416,18,77948
3,1004,App-1004,Mobile,Chen,Android,planning,green,2025-09-03,rebuild,110074,4,72068
4,1005,App-1005,Platform,Nguyen,Web,retired,green,2025-07-16,rehost,152662,18,84943


In [3]:
con = duckdb.connect(database=":memory:")
con.execute("CREATE TABLE apps AS SELECT * FROM df")
print(con.execute("SELECT COUNT(*) AS n FROM apps").fetchdf())

     n
0  120


In [4]:
def get_schema(con):
    tables = con.execute("SHOW TABLES").fetchall()
    schema = {}
    for (t,) in tables:
        cols = con.execute(f"DESCRIBE {t}").fetchdf()[["column_name", "column_type"]]
        schema[t] = cols
    return schema

schema = get_schema(con)
schema["apps"]

,column_name,column_type
0,app_id,BIGINT
1,app_name,VARCHAR
2,team,VARCHAR
3,director,VARCHAR
4,platform,VARCHAR
5,overall_status,VARCHAR
6,ryg_status,VARCHAR
7,planned_launch_date,VARCHAR
8,modernization_type,VARCHAR
9,monthly_active_users,BIGINT


In [5]:
ALLOWED_TABLES = {"apps"}
FORBIDDEN_KEYWORDS = {
    "INSERT","UPDATE","DELETE","DROP","ALTER","CREATE","TRUNCATE","GRANT","REVOKE",
    "COPY","ATTACH","DETACH","PRAGMA","CALL","EXEC","MERGE"
}

def is_read_only_select(sql: str) -> bool:
    parsed = sqlparse.parse(sql)
    if not parsed:
        return False
    stmt: Statement = parsed[0]
    first_kw = None
    for token in stmt.tokens:
        if token.ttype in Keyword:
            first_kw = token.value.upper()
            break
        if token.is_keyword:
            first_kw = token.value.upper()
            break
    return first_kw == "SELECT"

def contains_forbidden(sql: str) -> bool:
    up = re.sub(r"\s+", " ", sql.upper())
    return any(k in up for k in FORBIDDEN_KEYWORDS)

def references_only_allowed_tables(sql: str) -> bool:
    up = sql.upper()
    # very lightweight table detection (good enough for a demo)
    mentioned = set(re.findall(r"\bFROM\s+([A-Z_][A-Z0-9_]*)|\bJOIN\s+([A-Z_][A-Z0-9_]*)", up))
    flat = set()
    for a,b in mentioned:
        if a: flat.add(a.lower())
        if b: flat.add(b.lower())
    # If we couldn't detect tables, be conservative
    if len(flat) == 0:
        return False
    return flat.issubset(ALLOWED_TABLES)

def enforce_limit(sql: str, limit: int = 200) -> str:
    up = sql.upper()
    if "LIMIT" in up:
        return sql
    return sql.rstrip().rstrip(";") + f" LIMIT {limit};"

def validate_sql(sql: str) -> tuple[bool, str]:
    if not sql or not isinstance(sql, str):
        return False, "SQL empty."
    if contains_forbidden(sql):
        return False, "Forbidden SQL keyword detected."
    if not is_read_only_select(sql):
        return False, "Only SELECT queries are allowed."
    if not references_only_allowed_tables(sql):
        return False, "Query must reference only allowed tables."
    return True, "OK"